In [15]:
import numpy as np
import math
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

#Extract feature values and labels from the data
mnist_train_labels = np.array(mnist.train.labels)
mnist_train_images =  np.array(mnist.train.images)
mnist_valid_images =  np.array(mnist.validation.images)
mnist_valid_labels =  np.array(mnist.validation.labels)
mnist_test_labels =  np.array(mnist.test.labels)
mnist_test_images =  np.array(mnist.test.images)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [16]:
def pred_bayes_logistic(w_map, S_N, theta):
    mu_a = np.dot(w_map.T, theta)
    var_a = np.dot(np.dot(theta.T, S_N), theta)
    kappa_var = (1 + math.pi*var_a/8)^(-0.5)
    x = kappa_var*mu_a
    return 1/(1 + np.exp(-x))

In [17]:
#Hyper Parameter Acquisition
def hyper_para_bayes_logistic(m_0, S_0, Theta, y):
    w_map = m_0
    S_N = np.linalg.inv(S_0)
    Theta = Theta.T
    for i in range(Theta.shape[0]):
        S_N = S_N + y[i]*(1-y[i])*np.matmul(Theta[i].T, Theta[i])
    return w_map, S_N

In [18]:
class logistic_sigmoid_model:
    def _init(self):
        self.hyper_para_bayes_logistic = hyper_para_bayes_logistic
        self.pred_bayes_logistic = pred_bayes_logistic
    def training(self, Theta, y):
        self.w0 = np.random.normal(0, 1)
        self.S0 = np.diag(np.random.normal(0, 1, y.shape[0]))
    # Theta n*m (n samples, m features), y n*1
        self.w_map, self.S_N = self.hyper_para_bayes_logistic(self.w0, self.S0,Theta, y)
    def predict(self, theta):
        return self.pred_bayes_logistic(self.w_map, self.S_N, theta)

In [19]:
def multiclass_sigmoid_logistic(Theta, Y):
    n_class = Y.shape[1]
    models = []
    for i in range(n_class):
        models.append(logistic_sigmoid_model())
        models[i].training(Theta, Y[:, i])
    return models

In [11]:
## Prediction
def pred_multiclass_sigmoid_logistic(theta, Theta, Y):
    models = multiclass_sigmoid_logistic(Theta, Y)
    props = []
    for i in range(len(models)):
        props.append(models[i].predict(theta))
    return max(props)

In [ ]:
m_0 = 0.0532
S_0 = 0.7439
hyper_para_bayes_logistic(m_0, S_0, Theta, y)